In [2]:
from functions import read_data
from functions import NLP_utils

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/joaoprado/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
SAT_beers = read_data.fetch_satellite_df()
BA_beers = read_data.fetch_csv("../DATA/BeerAdvocate.tar",'beer')
RB_beers = read_data.fetch_csv("../DATA/RateBeer.tar",'beer')
BA_beers = BA_beers[BA_beers["nbr_ratings"] != 0].copy()
RB_beers = RB_beers[RB_beers["nbr_ratings"] != 0].copy()

# Keep only the maximum price, corresponding to the volume in the 'vol' column of the dataframe
SAT_beers["prix"] = SAT_beers['prix'].str.findall('[0-9]*\.?[0-9]*').apply(lambda x : max([float(price) for price in x if len(price)!=0]))
SAT_beers.head(15)


,nom,prix,url,visual,type,brasseur,from,pression,ephemere,alcool,vol
0,Brooklyn East IPA,6.0,details/P1/,/media/bieres/visuelsBoissons/Brookylnbrewery....,IPA,Brooklyn Brewery,USA,True,False,6.9,50
1,Cuvée des Trolls,6.0,details/P2/,/media/bieres/visuelsBoissons/cuveedestrolls.j...,Blonde,Brasserie Dubuisson,Belgique,True,False,7.0,50
2,Elvis Juice,6.0,details/P3/,/media/bieres/visuelsBoissons/ElvisJuice.jpg,IPA,Brewdog,Royaume-Uni,True,False,6.5,50
3,Grimbergen Blonde,6.0,details/P4/,/media/bieres/visuelsBoissons/grimbergen.jpeg,Blonde,Grimbergen,Belgique,True,False,6.7,50
4,Gurten Lager,4.5,details/P5/,/media/bieres/visuelsBoissons/Gurten-2.png,Blonde,Gurten,Suisse,True,False,4.8,50
5,Charles Quint Rubis,6.0,details/P6/,/media/bieres/visuelsBoissons/biere-charles-qu...,Ambrée,Haacht,Belgique,True,False,8.5,50
6,Lupulus Blonde,6.0,details/P7/,/media/bieres/visuelsBoissons/Lupulus.jpg,Blonde,Brasserie Lupulus,Belgique,True,False,8.5,50
7,O'Hara's Irish Stout,6.0,details/P8/,/media/bieres/visuelsBoissons/Oharas.jpeg,Stout,Carlow Brewing Company,Irelande,True,False,4.3,50
8,Schneider Weisse,6.0,details/P9/,/media/bieres/visuelsBoissons/Schneider_weisse...,Blanche,Weissen Bräuhaus G. Schneier & Sohn,Allemagne,True,False,5.4,50
9,Valaisanne Pale Ale,6.0,details/P10/,/media/bieres/visuelsBoissons/valaisanne-pale-...,Pale Ale,Brasserie Valaisanne,Suisse,True,False,5.2,50


In [4]:
sat_queries = NLP_utils.querify(SAT_beers)
#The queries should have the same document structure as the corpus. 
for index, biere in SAT_beers.iterrows():
    sat_queries[index] = (NLP_utils.tokenize(biere['nom']) +" "  + NLP_utils.tokenize(biere['brasseur']) + " " + NLP_utils.tokenize(str(biere['alcool'])))

In [5]:
#Take a lot of time unfortunately
BA_results = NLP_utils.vector_space_retrieval(sat_queries,BA_beers)
RB_results =  NLP_utils.vector_space_retrieval(sat_queries,RB_beers)
BA_results.columns = 'BA_' + BA_results.columns.values
RB_results.columns = 'RB_' + RB_results.columns.values

Documents tokenized
Documents tokenized


In [53]:
import pandas as pd

concat_result = pd.concat([BA_results.reset_index(),RB_results.reset_index()],axis=1)

concat_result.head(1)

,index,BA_beer_id,BA_beer_name,BA_brewery_id,BA_brewery_name,BA_style,BA_nbr_ratings,BA_nbr_reviews,BA_avg,BA_ba_score,...,RB_nbr_ratings,RB_overall_score,RB_style_score,RB_avg,RB_abv,RB_avg_computed,RB_zscore,RB_nbr_matched_valid_ratings,RB_avg_matched_valid_ratings,RB_similarity
0,119976,147,Brooklyn East India Pale Ale,45,Brooklyn Brewery,English India Pale Ale (IPA),2468,986,3.65,83.0,...,2229,77.0,55.0,3.4,6.9,3.394616,NaN,0,NaN,0.926097


In [54]:
from IPython.display import display_html 

results_df = NLP_utils.display_results_df(SAT_beers, concat_result,"nom",["BA_beer_name","RB_beer_name","BA_similarity","RB_similarity"])
results_styler = results_df[['nom',"BA_beer_name","RB_beer_name"]].style.apply(lambda x : NLP_utils.color(x,results_df[["BA_similarity","RB_similarity"]]),axis=None)
display_html(results_styler._repr_html_(),raw = True)

,nom,BA_beer_name,RB_beer_name
0,Brooklyn East IPA,Brooklyn East India Pale Ale,Brooklyn East IPA / India Pale Ale
1,Cuvée des Trolls,Cuvée Des Trolls,Dubuisson Cuvée des Trolls
2,Elvis Juice,Elvis Juice,Three Magnets Juice Juice Juice Storm Triple IPA
3,Grimbergen Blonde,Grimbergen Blonde,Grimbergen Blonde
4,Gurten Lager,Gurten BäreGold,Gurten Lager Hell
5,Charles Quint Rubis,Charles Quint Blonde / Keizer Karel Blond,Charles Quint Rouge Rubis / Keizer Karel Robijn Rood
6,Lupulus Blonde,Lupulus,Keltius / La Quince Humulus Lupulus Kince Lupulus
7,O'Hara's Irish Stout,O'Hara's Irish Stout,Carlow OHaras Irish Stout
8,Schneider Weisse,Schneider Weisse Tap 7 Unser Original,Schneider Weisse Original
9,Valaisanne Pale Ale,Valaisanne Lager,Valaisanne Pale Ale


In [37]:
SAT_beers.head(1)

,nom,prix,url,visual,type,brasseur,from,pression,ephemere,alcool,vol
0,Brooklyn East IPA,6.0,details/P1/,/media/bieres/visuelsBoissons/Brookylnbrewery....,IPA,Brooklyn Brewery,USA,True,False,6.9,50
